In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>, Cristovao Beirao da Cruz e Sliva <cberiaod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
import beamtest_analysis_helper as helper
import pandas as pd
import numpy as np
from glob import glob
import random
from tqdm import tqdm

In [ ]:
run_name = 'run12'
original_files = glob(f'./desy_TB_{run_name}/*feather')

board_ids_to_analyze = [0, 1, 3]
board_ids_to_drop = [2]
columns_to_read = ['evt', 'board', 'row', 'col', 'toa', 'tot', 'cal']
dut_id = 1
ref_id = 3

# columns_want_to_drop = [f'toa_{i}' for i in set(list_of_all_boards)-set(list_of_ignore_boards)]

columns_want_to_group = []
for i in board_ids_to_analyze:
    columns_want_to_group.append(f'row_{i}')
    columns_want_to_group.append(f'col_{i}')

In [ ]:
final_list = []
for isampling in tqdm(range(10)):
    files = random.sample(original_files, k=int(0.1*len(original_files)))

    last_evt = 0
    dataframes = []

    for idx, ifile in enumerate(files):
        tmp_df = pd.read_feather(ifile, columns=columns_to_read)

        if idx > 0:
            tmp_df['evt'] += last_evt
        last_evt += tmp_df['evt'].unique()[-1]

        ## Selecting good hits
        tdc_cuts = {}
        for idx in board_ids_to_analyze:
            # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
            if idx == 0:
                tdc_cuts[idx] = [tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]+50,  100, 500, 0, 600]
            else:
                tdc_cuts[idx] = [tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]-50, tmp_df.loc[tmp_df['board'] == idx]['cal'].mode()[0]+50,  0, 1100, 0, 600]

        filtered_df = helper.tdc_event_selection(tmp_df, tdc_cuts_dict=tdc_cuts)
        del tmp_df

        if filtered_df.empty:
            continue

        event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
        event_selection_col = None

        trig_selection = (event_board_counts[0] == 1)
        ref_selection = (event_board_counts[ref_id] == 1)
        event_selection_col = trig_selection & ref_selection

        selected_subset_df = filtered_df[filtered_df['evt'].isin(event_board_counts[event_selection_col].index)]
        selected_subset_df.reset_index(inplace=True, drop=True)

        dataframes.append(selected_subset_df)
        del event_board_counts, selected_subset_df, event_selection_col

    df = pd.concat(dataframes)
    df.reset_index(inplace=True, drop=True)
    del dataframes

    single_filtered_df = helper.singlehit_event_clear(df, ignore_boards=board_ids_to_drop)
    pivot_data_df = helper.making_pivot(single_filtered_df, 'evt', 'board', set({'board', 'evt', 'cal', 'tot'}), ignore_boards=board_ids_to_drop)
    del single_filtered_df

    min_hit_counter = 500*len(files)/len(original_files)
    # print('# of track cut:', min_hit_counter)
    combinations_df = pivot_data_df.groupby(columns_want_to_group).count()
    combinations_df['count'] = combinations_df['toa_0']
    # combinations_df.drop(columns_want_to_drop, axis=1, inplace=True)
    track_df = combinations_df.loc[combinations_df['count'] > min_hit_counter]
    track_df.reset_index(inplace=True)
    del pivot_data_df, combinations_df

    row_delta_TR = np.abs(track_df['row_0'] - track_df[f'row_{ref_id}']) <= 2
    row_delta_TD = np.abs(track_df['row_0'] - track_df[f'row_{dut_id}']) <= 1
    col_delta_TR = np.abs(track_df['col_0'] - track_df[f'col_{ref_id}']) <= 2
    col_delta_TD = np.abs(track_df['col_0'] - track_df[f'col_{dut_id}']) <= 1

    track_condition = (row_delta_TR) & (col_delta_TR) & (row_delta_TD) & (col_delta_TD)
    track_df = track_df[track_condition]

    final_list.append(track_df)
    del track_condition, track_df

In [ ]:
final_df = pd.concat(final_list)
final_df.drop(columns=['count'], inplace=True)
final_df = final_df.drop_duplicates(subset=columns_want_to_group, keep='first')
final_df.to_csv(f'{run_name}_good_track_candidates.csv', index=False)

del final_list, final_df